In [1]:
#PERMANENT CELL 1

import os
import mikeio
import mikeio1d
from mikeio1d.res1d import Res1D
import math
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ctypes
import traceback
MessageBox = ctypes.windll.user32.MessageBoxA
from Result_Lookup_Variables import *

In [3]:
for m in master_list:
    
#     model_area = m[0]
#     model = m[1]
    result_folder = m[2]
#     output_folder = m[3]
    result_list = m[4]    
    
    for r in result_list:
        header = r[0]
        file = r[1]
        res1d = Res1D(result_folder + '\\' + file)
        r.append(res1d)


In [239]:
result_folder = r"J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\Always_Latest_Master_Model_Simulations_FSA\Model"
node_df = pd.read_csv(result_folder + '\\ls_nodes.csv',dtype={'Node': str})
node_df.set_index('Node',inplace=True)
node_df['Asset Name'].fillna('',inplace=True)
pipe_df = pd.read_csv(result_folder + '\\ls_pipes.csv',dtype={'Pipe': str,'FromNode': str,'ToNode': str})
pipe_df.set_index('Pipe',inplace=True)

#Final product
Alternating rows:
Node, Link Upstream, Link Downstream, Node............Node

In [6]:
class Node():
    def __init__(self,node):    
        self.node = node
    def invert(self):        
        return node_df.loc[self.node,'Invert Level']
    def ground(self):        
        return node_df.loc[self.node,'Ground Level']
    def soh(self):        
        return node_df.loc[self.node,'Safe Operating Head']
    def asset_name(self):        
        return node_df.loc[self.node,'Asset Name']
    def cover(self):        
        return node_df.loc[self.node,'Cover Type']

class Pipe():
    def __init__(self,pipe):    
        self.pipe = pipe
    def height(self):        
        return pipe_df.loc[self.pipe,'Height'] 
    def uplevel(self):
        uplevel = pipe_df.loc[self.pipe,'UpLevel']
        if math.isnan(uplevel):
            node = pipe_df.loc[self.pipe,'FromNode']
            uplevel = node_df.loc[node,'Invert Level']
        return uplevel
    def dwlevel(self):
        dwlevel = pipe_df.loc[self.pipe,'DwLevel']
        if math.isnan(dwlevel):
            node = pipe_df.loc[self.pipe,'ToNode']
            dwlevel = node_df.loc[node,'Invert Level']
        return dwlevel
    

In [ ]:
Pipe('48502').uplevel()

In [9]:
r

['Nov 15 2021 Event',
 'FSA_WWF_2021-11-12_5d_2020pop_Base.res1d',
 'Nov 15 2021 Event',

In [ ]:
math.isnan(pipe_df.loc['48502','DwLevel'])

In [246]:
missing_pipes = []

node_delta_xs = [['US',-1],['DS',0]]

acronyms = list(pipe_df.Acronym.unique())
paths = []

loop_counter = 0

for acronym in acronyms:
    print(acronym)
    
    path_no = 0
    
    pipe_index_df = pipe_df[pipe_df.Acronym==acronym].copy()
    
    while len(pipe_index_df)>0:
             

        start_pipes = []
        for index, row in pipe_index_df.iterrows():
            if row['FromNode'] not in list(pipe_index_df.ToNode):

                start_pipes.append(index)

        start_pipes

        for start_pipe in start_pipes[:1]:
            exit_path = False
            chainage = 0

            node = pipe_index_df.loc[start_pipe,'FromNode']
            pipe = start_pipe

            path = []
            
            for node_delta_x in node_delta_xs:
                point = [path_no,0,acronym,'Node-'+node_delta_x[0],node,chainage+node_delta_x[1],Node(node).invert(),
                         Node(node).ground(),Node(node).soh(),Node(node).asset_name(),Node(node).cover(),
                        np.nan,np.nan]
                for r in result_list:
                    point.append(max(r[3].query.GetNodeValues(node, "WaterLevel")))
                path.append(point)

            i = 0
            while exit_path == False and i < 1000:

                point = [path_no,i*3+1,acronym,'Pipe US',pipe,chainage,Pipe(pipe).uplevel(),
                             Node(node).ground(),Node(node).soh(),Node(node).asset_name(),Node(node).cover(),
                            Pipe(pipe).height(),Pipe(pipe).height()+Pipe(pipe).uplevel()]

                for r in result_list:
                    point.append(max(r[3].query.GetReachStartValues(pipe, "WaterLevel")))
                path.append(point)

                chainage += pipe_index_df.loc[pipe,'Length']

                node = pipe_index_df.loc[pipe,'ToNode']


                point = [path_no,i*3+1,acronym,'Pipe DS',pipe,chainage-1,Pipe(pipe).dwlevel(),
                             Node(node).ground(),Node(node).soh(),Node(node).asset_name(),Node(node).cover(),
                            Pipe(pipe).height(),Pipe(pipe).height()+Pipe(pipe).dwlevel()]

                for r in result_list:
                    point.append(max(r[3].query.GetReachEndValues(pipe, "WaterLevel")))
                
                path.append(point)
                

                for node_delta_x in node_delta_xs:
                    point = [path_no,0,acronym,'Node-'+node_delta_x[0],node,chainage+node_delta_x[1],Node(node).invert(),
                                 Node(node).ground(),Node(node).soh(),Node(node).asset_name(),Node(node).cover(),
                                np.nan,np.nan]

                    for r in result_list:
                        point.append(max(r[3].query.GetNodeValues(node, "WaterLevel")))
                

                    path.append(point)

                pipe_index_df.drop(pipe,inplace=True)

                try:
                    pipe = pipe_index_df[pipe_index_df.FromNode==node].index[0]
                except:
                    exit_path = True   
                i += 1
                
                loop_counter += 1
                if loop_counter > 20000:
                    raise
                    
                
        paths += path
        
        path_no += 1

# paths
columns = ['Path Number','Sqn','Acronym','Type','MUID','Chainage','Invert Level','Ground Level','Safe Operating Head',
                                   'Asset Name','Cover Type','Height','Pipe Obvert']

for r in result_list:
    header = r[0]
    columns.append(header)
    

path_df=pd.DataFrame(paths,columns=columns)
       

SAT
SAC
PSM
GL2
GLC
GLK
PFM
NWF
POC
CQC
BNL
MHT
LZW
BNP
SAS
SGP
CFF
NWX
PSE
NWG
HTA
NWS
CQM
P2F
LKI
BNS
GLN
NWR
NCT
WMT
CQD
NWC
NWW
WDT
SAB
PSW
GLO
NRT
BIN
LZS
PIF
PIM
GLE
LZF
ML2
FRP
NXI
PSO
ALT
ALX
CQL
NZF
SFM
SPP
NWA
TSF
NXJ
NXG
NXF
STF
MLT
SYC
SMS
NWP
KYT
NPL
LZG
SAP
SAD
NWD
FST
PQO
NWT
HTB
LZT
NZE
NZD
PQE
NZA
NZB
NZI
GLB
WMW
NZC
BNC
NZH
NXE
PSS
BGO
MHO
NZG
NXP
TSI
VIN
WDO
NXA
PMA
SOF
QBF
GLT
BBX
NSM
NSD
NSX
M2A
RRT
MPK
BSX
RYO
NSA
MSO
NXK
BSW
NS4E1
NSS
NSF
BSI
MPR
BBL
NSP
MSW
NSR
NKR
MPY
NET
OPC
OP6
KZS
MIS
BBB
NSW
MSM
NXB
BBC
NKN
SSH
NXC
RYA
MPA
SSG
CTS
CI2
WFM
CB1
LTS
BBA
NNA
MS2W
RE2
SSS
SSC
EF3
KTZ
BYN
MRS
NS6
SSK
NSE
SSCH
SSJ
CV7
L2F
LAF
SSA
NFR
NDI
SSP
MST
SSF
N2T
VTH
SLA
LET
CZS
OSR
NS4
LAG
SST
RE1
CVS
SSD
BST
L1F
ROC
NGW
L3C
RO6
NTS
NNK
NKM
CIN
ROH
C2F
CV6
VT2F
SRC
BSE
NDR
CMP
LGL
LBG
OPA
CTO
SDD
WHR


In [96]:
with open(r'J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\Always_Latest_Master_Model_Simulations_FSA\Plot_Experiments\missing.mus', 'w') as f:
    f.write('msm_Link\n')
    for pipe in pipe_df.index:
        if not pipe in  path_df[path_df.Type!='Node'].MUID.unique():
            f.write(str(pipe) + '\n')
f.close()

In [277]:
output_folder = r'J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\Always_Latest_Master_Model_Simulations_FSA\Plot_Experiments\Long_Sections'

colors = ['black','black','green','red','brown','orange','grey','brown','midnightblue','tan','firebrick','darkturquoise',
         'darkorchid','dimgray','darkgoldenrod','indianred','deeppink']


for acronym in acronyms[:10]:
    
    
    path_acronym_df = path_df[path_df.Acronym==acronym]
    
    with open(output_folder + "\\Long_Section_" + acronym + ".html", 'w') as f:
        f.write('<link rel="stylesheet" href="..\Maps_And_CSS\style.css">\n')

        f.write('<!DOCTYPE html>\n')
        f.write('<html>\n')
        f.write('<head>\n')
        f.write('<meta charset="utf-8">\n')
        f.write('</head>\n')
        f.write('<body>\n')
        f.write('<div class="sidenav">\n')
        
        f.write("<h2>Map</h2>\n")                   
        map_string = '<img src="..\\Maps_And_CSS\\Acronym_' + acronym + '.jpg" alt="' + acronym + '">\n'
        f.write(map_string + "\n")
        f.write("<p></p>\n")
        
        f.write('</div>')
        f.write('<div class="main">\n')
        f.write("<h1>Long Sections for " + acronym + "</h1>\n")
  
        for path_no in path_acronym_df['Path Number'].unique():
            path_acronym_sub_df = path_acronym_df[path_acronym_df['Path Number']==path_no]
            path_acronym_node_df = path_acronym_sub_df[path_acronym_sub_df.Type=='Node-DS']

            fig = go.Figure()

            columns=['Invert Level','Pipe Obvert','Ground Level','Safe Operating Head']
            for r in result_list:
                header = r[0]
                columns.append(header)

            for i, column in enumerate(columns):
                if i < 4:
                    line_width = 4
                else:
                    line_width = 2
                fig.add_trace(go.Scatter(x=path_acronym_sub_df.Chainage, 
                                         y = path_acronym_sub_df[column],line_color=colors[i],name=column, line_width = line_width))

            lbl_x = []
            lbl_y = []
            lbl_text = []
            
            label_y = max(path_acronym_node_df['Ground Level']) + 2
            
            for index, row in path_acronym_node_df.iterrows():
                
                fig.add_shape(type="rect",x0=row['Chainage']-1, y0=row['Invert Level'], x1=row['Chainage'],
                              y1=row['Ground Level'],line=dict(color="RoyalBlue",width=1),)
                
                fig.add_shape(type="line",
                    x0=row['Chainage']-0.5, y0=row['Ground Level'], x1=row['Chainage']-0.5, y1=label_y-0.8,
                    line=dict(
                        color="Grey",
                        width=1,
                        dash="dot",
                    )
                )
                
                lbl_x.append(row['Chainage'] - 0.5)
                
                lbl_y.append(label_y)
                if row['Cover Type']==2:
                    cover_type = 'Sealed'
                else:
                    cover_type = 'Unsealed'
                lbl_text.append(str(row['Asset Name']) + '<br>' +  str(row['MUID']) + '<br>' + cover_type)
                
            fig.add_trace(go.Scatter(
                x=lbl_x,
                y=lbl_y,
                text=lbl_text,
                mode="text",
                name="Labels"
            ))
            
            fig.update_layout(
                autosize=False,
                width = 1362,
                height=700,
                margin=dict(
                    l=50,
                    r=50,
                    b=25,
                    t=35,
                    pad=4
                    ),
                yaxis_title='Geodetic Elevation(m)',
                xaxis_title='Chainage (m)'
                )

            f.write("<h2>Long Section " + str(path_no+1) + "</h2>\n")
            f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
            
        f.write('</div>')
        f.write('</body>')
        f.write('</html>')
        f.close()

In [264]:
lbl_y = []
if (index % 2) == 0:
                    lbl_y.append(label_y)
                else:
                    lbl_y.append(label_y + 2)

[12.27,
 12.27,
 12.27,
 12.27,
 12.27,
 12.27,
 12.27,
 12.27,
 12.27,
 12.27,
 12.27,
 12.27,
 12.27,
 12.27,
 12.27,
 12.27,
 12.27,
 12.27,
 12.27,
 12.27,
 12.27,
 12.27,
 12.27,
 12.27]

In [186]:
path_df[path_df.Acronym=='SYC'].head(50)

,Path Number,Sqn,Acronym,Type,MUID,Chainage,Invert Level,Ground Level,Safe Operating Head,Asset Name,Cover Type,Height,Pipe Obvert,2yr 24h No Climate Change,5yr 24h No Climate Change,10yr 24h No Climate Change,Dry Weather Flow,Jan 31 2020 Event,Nov 15 2021 Event
5358,0,0,SYC,Node-US,DON_SCSO_EastGate,-1.000000,21.89,24.27,22.969999,MH8,2,NaN,NaN,21.99880,22.00217,22.00468,21.982470,22.00580,22.01385
5359,0,0,SYC,Node-DS,DON_SCSO_EastGate,0.000000,21.89,24.27,22.969999,MH8,2,NaN,NaN,21.99880,22.00217,22.00468,21.982470,22.00580,22.01385
5360,0,1,SYC,Pipe US,SCS_Outlet_Pipe1,0.000000,21.89,24.27,22.969999,MH8,2,0.45,22.34,21.99880,22.00217,22.00468,21.982470,22.00580,22.01385
5361,0,1,SYC,Pipe DS,SCS_Outlet_Pipe1,61.289185,19.85,24.58,21.340000,MH7,2,0.45,20.30,20.14397,20.15456,20.16235,20.098660,20.16517,20.19979
5362,0,0,SYC,Node-US,7245,61.289185,19.85,24.58,21.340000,MH7,2,NaN,NaN,20.14397,20.15456,20.16235,20.098660,20.16517,20.19979
5363,0,0,SYC,Node-DS,7245,62.289185,19.85,24.58,21.340000,MH7,2,NaN,NaN,20.14397,20.15456,20.16235,20.098660,20.16517,20.19979
5364,0,4,SYC,Pipe US,43400,62.289185,19.85,24.58,21.340000,MH7,2,0.45,20.30,20.01208,20.01864,20.02351,19.984790,20.02527,20.04688
5365,0,4,SYC,Pipe DS,43400,123.705477,19.05,20.61,20.110000,MH6,3,0.45,19.50,19.40220,19.41480,19.42385,19.347250,19.42696,19.46303
5366,0,0,SYC,Node-US,7246,123.705477,19.05,20.61,20.110000,MH6,3,NaN,NaN,19.40220,19.41480,19.42385,19.347250,19.42696,19.46303
5367,0,0,SYC,Node-DS,7246,124.705477,19.05,20.61,20.110000,MH6,3,NaN,NaN,19.40220,19.41480,19.42385,19.347250,19.42696,19.46303


In [190]:
node_df

AttributeError: 'DataFrame' object has no attribute 'Index'

In [197]:
node_df.loc['7244']

Asset Name              MH21
Cover Type                 3
Network Type               1
Ground Level           42.56
Invert Level           38.22
Safe Operating Head    42.06
Name: 7244, dtype: object

In [231]:
path_acronym_node_df['Asset Name'].fillna('',inplace=True)
path_acronym_node_df

C:\Users\hloecke\AppData\Local\Temp\ipykernel_31552\3411030764.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Path Number,Sqn,Acronym,Type,MUID,Chainage,Invert Level,Ground Level,Safe Operating Head,Asset Name,Cover Type,Height,Pipe Obvert,2yr 24h No Climate Change,5yr 24h No Climate Change,10yr 24h No Climate Change,Dry Weather Flow,Jan 31 2020 Event,Nov 15 2021 Event
15635,0,0,SDD,Node-DS,22216,0.0,0.34,4.77,NaN,,2,NaN,NaN,4.838267,5.769511,6.191171,2.834573,5.848464,6.206648
15639,0,0,SDD,Node-DS,1397,1.1,0.34,4.77,6.74,MH1A,2,NaN,NaN,4.862594,5.801842,6.227404,3.098386,5.887120,6.232188
15643,0,0,SDD,Node-DS,22217,2.2,0.34,4.77,NaN,,2,NaN,NaN,4.857595,5.793857,6.217516,2.816636,5.873017,6.230649


In [238]:
for mh in path_acronym_node_df['Asset Name']:
    if mh == None:
        print('Nooo')
    else:
        print(mh)


MH1A



In [241]:
node_df.head(50)

,Asset Name,Cover Type,Network Type,Ground Level,Invert Level,Safe Operating Head
Node,,,,,,
1_1129_N,,2,1,3.50,-0.46,NaN
1_1130_N,,2,1,0.30,-0.46,NaN
1_1131_N,,2,1,0.62,-0.14,NaN
1_1132_N,,2,1,1.54,0.78,NaN
1_1133_N,,2,1,1.63,0.87,NaN
1_1137_N,,2,1,2.46,1.70,NaN
1_1140_N,,2,1,2.56,1.80,NaN
1_1141_N,,2,1,1.26,0.50,NaN
1_1142_N,,2,1,0.68,-0.08,NaN


In [244]:
path_df[path_df.Acronym=='NXA']

,Path Number,Sqn,Acronym,Type,MUID,Chainage,Invert Level,Ground Level,Safe Operating Head,Asset Name,Cover Type,Height,Pipe Obvert,2yr 24h No Climate Change,5yr 24h No Climate Change,10yr 24h No Climate Change,Dry Weather Flow,Jan 31 2020 Event,Nov 15 2021 Event


In [245]:
pipe_df[pipe_df.Acronym=='NXA']

,Acronym,Height,Length,UpLevel,DwLevel,FromNode,ToNode
Pipe,,,,,,,
47944,NXA,0.45,27.552394,NaN,NaN,NW_20th,7929
